In [1]:
import pandas as pd
from pandas import Series, DataFrame
import matplotlib.pyplot as plt
import numpy as np
import os
import re
from urllib.request import urlopen 


In [2]:
catalog_Galaxy_ZooAUG  = pd.read_csv ('D:/nir/gc/GalaxyZoo1_DR_table2.csv ')

In [3]:
class ImageLoader:
    def __init__(self, download_path, galaxy_data):
        self.download_path = download_path
        self.galaxy_data = galaxy_data

    def parse_coordinates(self, coordinate_str):
        parts = coordinate_str.split(':')
        return "_".join(parts)

    def getDeg(self, hours, minutes, seconds):
        DegSeconds = seconds * 15
        DegMinutes = minutes * 15
        Deg = hours * 15
        return Deg + 0.0167 * DegMinutes + 0.000278 * DegSeconds

    def sign(self, value):
        if value >= 0:
            return 1
        return -1

    def load_image(self, RA, DEC, spiral, elliptical, uncertain):
        path = self.download_path
        RA_str = self.parse_coordinates(str(RA))
        DEC_str = self.parse_coordinates(str(DEC))
        
        galaxy_type = ""
        if spiral == 1:
            galaxy_type = "SPIRAL"
        elif elliptical == 1:
            galaxy_type = "ELLIPTICAL"
        elif uncertain == 1:
            galaxy_type = "UNCERTAIN"
        
        filename = f"{RA_str}_{DEC_str}_{galaxy_type}.jpg"
        full_path = os.path.join(path, re.sub(r':', '_', filename))

        resource = urlopen("https://skyserver.sdss.org/dr16/SkyServerWS/ImgCutout/getjpeg?TaskName=Skyserver.Chart.ShowNearest&ra=" + str(RA) + "&dec=" + str(DEC) + "&scale=0.2&opt=")
        
        with open(full_path, 'wb') as outFile:
            outFile.write(resource.read())

In [ ]:
spirals = catalog_Galaxy_ZooAUG[catalog_Galaxy_ZooAUG['SPIRAL'] == 1].head(14000)
ellipticals = catalog_Galaxy_ZooAUG[catalog_Galaxy_ZooAUG['ELLIPTICAL'] == 1].head(14000)
uncertains = catalog_Galaxy_ZooAUG[catalog_Galaxy_ZooAUG['UNCERTAIN'] == 1].head(14000)

scraper = ImageLoader(download_path='D:/nir/gc/imagesBalance', galaxy_data=catalog_Galaxy_ZooAUG)

for index, galaxy in spirals.iterrows():
    try:
        scraper.load_image(galaxy['RA'], galaxy['DEC'], 1, 0, 0)
    except Exception as e:
        print(f"Ошибка при загрузке изображения для галактики {index}: {e}")


for index, galaxy in uncertains.iterrows():
    try:
        scraper.load_image(galaxy['RA'], galaxy['DEC'], 0, 0, 1)
    except Exception as e:
        print(f"Ошибка при загрузке изображения для галактики {index}: {e}")


for index, galaxy in ellipticals.iterrows():
    try:
        scraper.load_image(galaxy['RA'], galaxy['DEC'], 0, 1, 0)
    except Exception as e:
        print(f"Ошибка при загрузке изображения для галактики {index}: {e}")

print(f"Загрузка завершена")


In [ ]:
import shutil

base_path = 'D:/nir/gc/imagesBalance'
output_dir = 'D:/nir/gc/sorted_images'


class_dirs = ['SPIRAL', 'ELLIPTICAL', 'UNCERTAIN']
for class_dir in class_dirs:
    os.makedirs(os.path.join(output_dir, class_dir), exist_ok=True)


for filename in os.listdir(base_path):
    if filename.endswith('.jpg'):
        src_path = os.path.join(base_path, filename) 

        if 'SPIRAL' in filename:
            dst_path = os.path.join(output_dir, 'SPIRAL', filename)
        elif 'ELLIPTICAL' in filename:
            dst_path = os.path.join(output_dir, 'ELLIPTICAL', filename)
        elif 'UNCERTAIN' in filename:
            dst_path = os.path.join(output_dir, 'UNCERTAIN', filename)
        else:
            continue 

        shutil.copy(src_path, dst_path) 

print("Копирование завершено")


# Создание аугментированных наборов данных

In [ ]:
from PIL import Image
from skimage.util import random_noise

base_path = r'D:\nir\gc\sorted_images42000' 
output_base = r'D:\nir\gc\augmented_dataT' 
categories = ['SPIRAL', 'ELLIPTICAL', 'UNCERTAIN']


augmentations = {
    "original_21000": ("orig_", lambda img: img),
    "noise_21000": ("noise_", lambda img: Image.fromarray((random_noise(np.array(img), mode='gaussian', var=0.01) * 255).astype(np.uint8))),
    "horizontal_flip_21000": ("hflip_", lambda img: img.transpose(Image.FLIP_LEFT_RIGHT)),
    "vertical_flip_21000": ("vflip_", lambda img: img.transpose(Image.FLIP_TOP_BOTTOM)),
    "rotate_90_cw_21000": ("rot90cw_", lambda img: img.rotate(-90, expand=True)),
    "rotate_90_ccw_21000": ("rot90ccw_", lambda img: img.rotate(90, expand=True)),
    "rotate_15_cw_21000": ("rot15cw_", lambda img: img.rotate(-15, expand=True)),
    "rotate_15_ccw_21000": ("rot15ccw_", lambda img: img.rotate(15, expand=True))
}

# Создание папок
for aug_name in augmentations:
    for category in categories:
        os.makedirs(os.path.join(output_base, aug_name, category), exist_ok=True)

# Применение аугментации и сохранение файлов
for category in categories:
    category_path = os.path.join(base_path, category)
    files = [f for f in os.listdir(category_path) if f.endswith('.jpg')][:7000]  

    for file_name in files:
        img_path = os.path.join(category_path, file_name)
        img = Image.open(img_path)

        for aug_name, (prefix, aug_func) in augmentations.items():
            aug_img = aug_func(img)

            base_name, ext = os.path.splitext(file_name)  
            new_file_name = f"{prefix}{base_name}{ext}" 
            save_path = os.path.join(output_base, aug_name, category, new_file_name)
            aug_img.save(save_path)

print("Создание аугментированных наборов завершено")

In [16]:
import os
import shutil

original_path = r'D:\nir\gc\augmented_data\original_21000' 
augmented_base_path = r'D:\nir\gc\augmented_data' 
output_base = r'D:\nir\gc\final_combined_data'  

combined_folders = {
    "original_and_noise_42000": "noise_21000",
    "original_and_vertical_flip_42000": "vertical_flip_21000",
    "original_and_horizontal_flip_42000": "horizontal_flip_21000",
    "original_and_rotate_90_cw_42000": "rotate_90_cw_21000",
    "original_and_rotate_90_ccw_42000": "rotate_90_ccw_21000",
    "original_and_rotate_15_cw_42000": "rotate_15_cw_21000",
    "original_and_rotate_15_ccw_42000": "rotate_15_ccw_21000"
}

categories = ['SPIRAL', 'ELLIPTICAL', 'UNCERTAIN']

for combined_name in combined_folders:
    for category in categories:
        os.makedirs(os.path.join(output_base, combined_name, category), exist_ok=True)

for combined_name, aug_folder in combined_folders.items():
    for category in categories:
        orig_src = os.path.join(original_path, category)
        aug_src = os.path.join(augmented_base_path, aug_folder, category)
        dest_folder = os.path.join(output_base, combined_name, category)

        for file_name in os.listdir(orig_src):
            shutil.copy(os.path.join(orig_src, file_name), os.path.join(dest_folder, file_name))

        for file_name in os.listdir(aug_src):
            shutil.copy(os.path.join(aug_src, file_name), os.path.join(dest_folder, file_name))

print("Создание комбинированных папок завершено")


Создание комбинированных папок завершено


In [17]:
import os
import shutil

original_path = r'D:\nir\gc\augmented_dataT\original_21000'  
augmented_base_path = r'D:\nir\gc\augmented_data'  
output_base = r'D:\nir\gc\final_mixed_data_42000'  

augmentations = {
    "original": "original_21000",
    "noise": "noise_21000",
    "horizontal_flip": "horizontal_flip_21000",
    "vertical_flip": "vertical_flip_21000",
    "rotate_90_cw": "rotate_90_cw_21000",
    "rotate_90_ccw": "rotate_90_ccw_21000",
    "rotate_15_cw": "rotate_15_cw_21000",
    "rotate_15_ccw": "rotate_15_ccw_21000"
}

categories = ['SPIRAL', 'ELLIPTICAL', 'UNCERTAIN']

for category in categories:
    os.makedirs(os.path.join(output_base, category), exist_ok=True)

for aug_name, folder in augmentations.items():
    total_copied = {category: 0 for category in categories} 

    for category in categories:
        src_folder = os.path.join(augmented_base_path, folder, category)
        dest_folder = os.path.join(output_base, category)

        files = [f for f in os.listdir(src_folder) if f.endswith('.jpg')][:1750]

        for file_name in files:
            shutil.copy(os.path.join(src_folder, file_name), os.path.join(dest_folder, file_name))
            total_copied[category] += 1 

    print(f"{aug_name}: {total_copied} файлов скопировано")

final_counts = {category: len(os.listdir(os.path.join(output_base, category))) for category in categories}
print(f"ИТОГОВОЕ РАСПРЕДЕЛЕНИЕ: {final_counts}")

print("Создание финальной папки с 42000 изображениями завершено")


original: {'SPIRAL': 10, 'ELLIPTICAL': 10, 'UNCERTAIN': 10} файлов скопировано
noise: {'SPIRAL': 10, 'ELLIPTICAL': 10, 'UNCERTAIN': 10} файлов скопировано
horizontal_flip: {'SPIRAL': 10, 'ELLIPTICAL': 10, 'UNCERTAIN': 10} файлов скопировано
vertical_flip: {'SPIRAL': 10, 'ELLIPTICAL': 10, 'UNCERTAIN': 10} файлов скопировано
rotate_90_cw: {'SPIRAL': 10, 'ELLIPTICAL': 10, 'UNCERTAIN': 10} файлов скопировано
rotate_90_ccw: {'SPIRAL': 10, 'ELLIPTICAL': 10, 'UNCERTAIN': 10} файлов скопировано
rotate_15_cw: {'SPIRAL': 10, 'ELLIPTICAL': 10, 'UNCERTAIN': 10} файлов скопировано
rotate_15_ccw: {'SPIRAL': 10, 'ELLIPTICAL': 10, 'UNCERTAIN': 10} файлов скопировано
ИТОГОВОЕ РАСПРЕДЕЛЕНИЕ: {'SPIRAL': 80, 'ELLIPTICAL': 80, 'UNCERTAIN': 80}
Создание финальной папки с 42000 изображениями завершено!


# Определения оптимального количества эпох обучения

In [ ]:
import os
import time 
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from sklearn.metrics import classification_report


DATASET_PATH = 'D:/nir/gc/sorted_images42000' 
MODEL_SAVE_PATH = 'galaxy_classifier_original.keras' 


IMG_SIZE = (128, 128)
BATCH_SIZE = 32
EPOCHS = 50


if not os.path.exists(DATASET_PATH):
    raise FileNotFoundError(f"Указанная директория '{DATASET_PATH}' не найдена.")


datagen = ImageDataGenerator(
    rescale=1.0/255.0,
    validation_split=0.2
)


train_generator = datagen.flow_from_directory(
    directory=DATASET_PATH,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=True,
    subset='training'
)


val_generator = datagen.flow_from_directory(
    directory=DATASET_PATH,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False,
    subset='validation'
)


model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(3, activation='softmax') 
])


model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)


model.summary()




start_time = time.time()


history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=EPOCHS
)


end_time = time.time()


training_time = (end_time - start_time) / 60  
print(f"\nВремя обучения: {training_time:.2f} минут")


val_generator.reset()
predictions = model.predict(val_generator, steps=val_generator.samples // BATCH_SIZE + 1)
predicted_classes = np.argmax(predictions, axis=1)  
true_classes = val_generator.classes 
class_labels = list(val_generator.class_indices.keys()) 


report = classification_report(true_classes, predicted_classes, target_names=class_labels)
print("\nОтчёт о классификации:")
print(report)


model.save(MODEL_SAVE_PATH)
print(f"\nМодель сохранена в {MODEL_SAVE_PATH}")


In [20]:
import pickle

with open('training_history.pkl', 'wb') as f:
    pickle.dump(history.history, f)


In [7]:
import pickle

with open('training_history3.pkl', 'rb') as f:
    loaded_history_ = pickle.load(f)

In [ ]:
import matplotlib.pyplot as plt

plt.plot(loaded_history_['loss'], label='Потери на обучающей выборке')

plt.xlabel('Эпоха')
plt.ylabel('Функция потерь')

plt.ylim(0, 1.0)
plt.xlim(0, 50.0)

plt.legend()

plt.savefig('training_loss_plot_.png', dpi=300, bbox_inches='tight')

plt.show()


# Модель для обучения 

In [ ]:
import os
import time  
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from sklearn.metrics import classification_report

DATASET_PATH = 'D:/nir/gc/sorted42000' 
MODEL_SAVE_PATH = 'galaxy_classifier_original.keras' 

IMG_SIZE = (128, 128)
BATCH_SIZE = 32
EPOCHS = 20

if not os.path.exists(DATASET_PATH):
    raise FileNotFoundError(f"Указанная директория '{DATASET_PATH}' не найдена.")

datagen = ImageDataGenerator(
    rescale=1.0/255.0,
    validation_split=0.2
)

train_generator = datagen.flow_from_directory(
    directory=DATASET_PATH,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=True,
    subset='training'
)


val_generator = datagen.flow_from_directory(
    directory=DATASET_PATH,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False,
    subset='validation'
)


model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5), 
    Dense(3, activation='softmax')  
])

model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

model.summary()



start_time = time.time()


history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=EPOCHS
)


end_time = time.time()


training_time = (end_time - start_time) / 60  
print(f"\nВремя обучения: {training_time:.2f} минут")


val_generator.reset()
predictions = model.predict(val_generator, steps=val_generator.samples // BATCH_SIZE + 1)
predicted_classes = np.argmax(predictions, axis=1)  
true_classes = val_generator.classes  
class_labels = list(val_generator.class_indices.keys())  


report = classification_report(true_classes, predicted_classes, target_names=class_labels)
print("\nОтчёт о классификации:")
print(report)


model.save(MODEL_SAVE_PATH)
print(f"\nМодель сохранена в {MODEL_SAVE_PATH}")


# Взаимодействующие галактики

In [4]:
spirals = catalog_Galaxy_ZooAUG[catalog_Galaxy_ZooAUG['SPIRAL'] == 1] 
ellipticals = catalog_Galaxy_ZooAUG[catalog_Galaxy_ZooAUG['ELLIPTICAL'] == 1]  
uncertains = catalog_Galaxy_ZooAUG[catalog_Galaxy_ZooAUG['UNCERTAIN'] == 1]  

def count_mergers(df):
    return (df['P_MG'] > 0.4).sum()

spiral_mergers = count_mergers(spirals)
elliptical_mergers = count_mergers(ellipticals)
uncertain_mergers = count_mergers(uncertains)

spiral_total = len(spirals)
elliptical_total = len(ellipticals)
uncertain_total = len(uncertains)

spiral_merger_percent = (spiral_mergers / spiral_total) * 100 if spiral_total > 0 else 0
elliptical_merger_percent = (elliptical_mergers / elliptical_total) * 100 if elliptical_total > 0 else 0
uncertain_merger_percent = (uncertain_mergers / uncertain_total) * 100 if uncertain_total > 0 else 0

print(f"Спиральные галактики: {spiral_total}, с merger: {spiral_mergers} ({spiral_merger_percent:.2f}%)")
print(f"Эллиптические галактики: {elliptical_total}, с merger: {elliptical_mergers} ({elliptical_merger_percent:.2f}%)")
print(f"Неправильные галактики: {uncertain_total}, с merger: {uncertain_mergers} ({uncertain_merger_percent:.2f}%)")


Спиральные галактики: 190225, с merger: 0 (0.00%)
Эллиптические галактики: 62190, с merger: 0 (0.00%)
Неправильные галактики: 415529, с merger: 10790 (2.60%)


In [16]:
class ImageLoader:
    def __init__(self, download_path, galaxy_data):
        self.download_path = download_path
        self.galaxy_data = galaxy_data

    def parse_coordinates(self, coordinate_str):
        parts = coordinate_str.split(':')
        return "_".join(parts)

    def getDeg(self, hours, minutes, seconds):
        DegSeconds = seconds * 15
        DegMinutes = minutes * 15
        Deg = hours * 15
        return Deg + 0.0167 * DegMinutes + 0.000278 * DegSeconds

    def sign(self, value):
        return 1 if value >= 0 else -1

    def load_image(self, RA, DEC, spiral, elliptical, uncertain, merger):
        path = self.download_path
        RA_str = self.parse_coordinates(str(RA))
        DEC_str = self.parse_coordinates(str(DEC))

        # Определение типа галактики
        galaxy_type = ""
        if spiral == 1:
            galaxy_type = "SPIRAL"
        elif elliptical == 1:
            galaxy_type = "ELLIPTICAL"
        elif uncertain == 1:
            galaxy_type = "UNCERTAIN"
        elif merger == 1:
            galaxy_type = "MERGER"

        filename = f"{RA_str}_{DEC_str}_{galaxy_type}.jpg"
        full_path = os.path.join(path, re.sub(r':', '_', filename))

        try:
            resource = urlopen(
                f"https://skyserver.sdss.org/dr16/SkyServerWS/ImgCutout/getjpeg?"
                f"TaskName=Skyserver.Chart.ShowNearest&ra={RA}&dec={DEC}&scale=0.2&opt="
            )

            with open(full_path, 'wb') as outFile:
                outFile.write(resource.read())

        except Exception as e:
            print(f"Ошибка при загрузке изображения ({RA}, {DEC}): {e}")

In [ ]:
spirals = catalog_Galaxy_ZooAUG[catalog_Galaxy_ZooAUG['SPIRAL'] == 1].head(10500)
ellipticals = catalog_Galaxy_ZooAUG[catalog_Galaxy_ZooAUG['ELLIPTICAL'] == 1].head(10500)

uncertains = catalog_Galaxy_ZooAUG[
    (catalog_Galaxy_ZooAUG['UNCERTAIN'] == 1) & (catalog_Galaxy_ZooAUG['P_MG'] < 0.4)
].head(10500)

mergers = catalog_Galaxy_ZooAUG[
    (catalog_Galaxy_ZooAUG['UNCERTAIN'] == 1) & (catalog_Galaxy_ZooAUG['P_MG'] > 0.4)
].head(10500)

scraper = ImageLoader(download_path='D:/nir/gc/images_with_merge', galaxy_data=catalog_Galaxy_ZooAUG)
loaded_images = 0

def load_galaxy_images(df, label, spiral, elliptical, uncertain, merger):
    global loaded_images
    for index, galaxy in df.iterrows():
        try:
            scraper.load_image(galaxy['RA'], galaxy['DEC'], spiral, elliptical, uncertain, merger)
            loaded_images += 1
        except Exception as e:
            print(f"Ошибка при загрузке изображения для галактики {index}: {e}")

    print(f"Загружено {loaded_images} {label} галактик")

load_galaxy_images(mergers, "взаимодействующих", 0, 0, 0, 1)
load_galaxy_images(uncertains, "неправильных", 0, 0, 1, 0)
load_galaxy_images(spirals, "спиральных", 1, 0, 0, 0)
load_galaxy_images(ellipticals, "эллиптических", 0, 1, 0, 0)

In [25]:
import shutil
import os

base_path = 'D:/nir/gc/images_with_merge'
output_dir = 'D:/nir/gc/images_merge'

class_dirs = ['SPIRAL', 'ELLIPTICAL', 'UNCERTAIN', 'MERGER']
for class_dir in class_dirs:
    os.makedirs(os.path.join(output_dir, class_dir), exist_ok=True)

for filename in os.listdir(base_path):
    if filename.endswith('.jpg'):
        src_path = os.path.join(base_path, filename) 

        if 'SPIRAL' in filename:
            dst_path = os.path.join(output_dir, 'SPIRAL', filename)
        elif 'ELLIPTICAL' in filename:
            dst_path = os.path.join(output_dir, 'ELLIPTICAL', filename)
        elif 'UNCERTAIN' in filename:
            dst_path = os.path.join(output_dir, 'UNCERTAIN', filename)
        elif 'MERGER' in filename:
            dst_path = os.path.join(output_dir, 'MERGER', filename)
        else:
            continue 

        shutil.copy(src_path, dst_path) 

print("Копирование завершено")

Копирование завершено


# Создание аугментированных наборов данных

In [33]:
from PIL import Image
from skimage.util import random_noise
import os
import numpy as np

base_path = r'D:\nir\gc\images_merge' 
output_base = r'D:\nir\gc\augmented_data_with_merge' 
categories = ['SPIRAL', 'ELLIPTICAL', 'UNCERTAIN', 'MERGER']

augmentations = {
    "original_21000": ("orig_", lambda img: img),
    "noise_21000": ("noise_", lambda img: Image.fromarray((random_noise(np.array(img), mode='gaussian', var=0.01) * 255).astype(np.uint8))),
    "horizontal_flip_21000": ("hflip_", lambda img: img.transpose(Image.FLIP_LEFT_RIGHT)),
    "vertical_flip_21000": ("vflip_", lambda img: img.transpose(Image.FLIP_TOP_BOTTOM)),
    "rotate_90_cw_21000": ("rot90cw_", lambda img: img.rotate(-90, expand=True)),
    "rotate_90_ccw_21000": ("rot90ccw_", lambda img: img.rotate(90, expand=True)),
    "rotate_15_cw_21000": ("rot15cw_", lambda img: img.rotate(-15, expand=True)),
    "rotate_15_ccw_21000": ("rot15ccw_", lambda img: img.rotate(15, expand=True))
}

for aug_name in augmentations:
    for category in categories:
        os.makedirs(os.path.join(output_base, aug_name, category), exist_ok=True)

for category in categories:
    category_path = os.path.join(base_path, category)
    files = [f for f in os.listdir(category_path) if f.endswith('.jpg')][:7000]  

    for file_name in files:
        img_path = os.path.join(category_path, file_name)
        img = Image.open(img_path)

        for aug_name, (prefix, aug_func) in augmentations.items():
            aug_img = aug_func(img)

            base_name, ext = os.path.splitext(file_name)  
            new_file_name = f"{prefix}{base_name}{ext}" 
            save_path = os.path.join(output_base, aug_name, category, new_file_name)
            aug_img.save(save_path)

print("Создание аугментированных наборов завершено")

Создание аугментированных наборов завершено


In [34]:
original_path = r'D:\nir\gc\augmented_data_with_merge\original_21000' 
augmented_base_path = r'D:\nir\gc\augmented_data_with_merge' 
output_base = r'D:\nir\gc\final_combined_data_with_merge'  

combined_folders = {
    "original_and_noise_42000": "noise_21000",
    "original_and_vertical_flip_42000": "vertical_flip_21000",
    "original_and_horizontal_flip_42000": "horizontal_flip_21000",
    "original_and_rotate_90_cw_42000": "rotate_90_cw_21000",
    "original_and_rotate_90_ccw_42000": "rotate_90_ccw_21000",
    "original_and_rotate_15_cw_42000": "rotate_15_cw_21000",
    "original_and_rotate_15_ccw_42000": "rotate_15_ccw_21000"
}


categories = ['SPIRAL', 'ELLIPTICAL', 'UNCERTAIN', 'MERGER']

for combined_name in combined_folders:
    for category in categories:
        os.makedirs(os.path.join(output_base, combined_name, category), exist_ok=True)

for combined_name, aug_folder in combined_folders.items():
    for category in categories:
        orig_src = os.path.join(original_path, category)
        aug_src = os.path.join(augmented_base_path, aug_folder, category)
        dest_folder = os.path.join(output_base, combined_name, category)

        for file_name in os.listdir(orig_src):
            shutil.copy(os.path.join(orig_src, file_name), os.path.join(dest_folder, file_name))

        for file_name in os.listdir(aug_src):
            shutil.copy(os.path.join(aug_src, file_name), os.path.join(dest_folder, file_name))

print("Создание комбинированных папок завершено")

Создание комбинированных папок завершено


In [35]:
original_path = r'D:\nir\gc\augmented_data_with_merge\original_21000'  
augmented_base_path = r'D:\nir\gc\augmented_data_with_merge'  
output_base = r'D:\nir\gc\final_mixed_data__with_merge'  

augmentations = {
    "original": "original_21000",
    "noise": "noise_21000",
    "horizontal_flip": "horizontal_flip_21000",
    "vertical_flip": "vertical_flip_21000",
    "rotate_90_cw": "rotate_90_cw_21000",
    "rotate_90_ccw": "rotate_90_ccw_21000",
    "rotate_15_cw": "rotate_15_cw_21000",
    "rotate_15_ccw": "rotate_15_ccw_21000"
}

categories = ['SPIRAL', 'ELLIPTICAL', 'UNCERTAIN', 'MERGER']

for category in categories:
    os.makedirs(os.path.join(output_base, category), exist_ok=True)

for aug_name, folder in augmentations.items():
    total_copied = {category: 0 for category in categories} 

    for category in categories:
        src_folder = os.path.join(augmented_base_path, folder, category)
        dest_folder = os.path.join(output_base, category)

        files = [f for f in os.listdir(src_folder) if f.endswith('.jpg')][:1313]

        for file_name in files:
            shutil.copy(os.path.join(src_folder, file_name), os.path.join(dest_folder, file_name))
            total_copied[category] += 1 

    print(f"{aug_name}: {total_copied} файлов скопировано")

final_counts = {category: len(os.listdir(os.path.join(output_base, category))) for category in categories}
print(f"ИТОГОВОЕ РАСПРЕДЕЛЕНИЕ: {final_counts}")

print("Создание финальной папки с 42000 изображениями завершено")

original: {'SPIRAL': 1313, 'ELLIPTICAL': 1313, 'UNCERTAIN': 1313, 'MERGER': 1313} файлов скопировано
noise: {'SPIRAL': 1313, 'ELLIPTICAL': 1313, 'UNCERTAIN': 1313, 'MERGER': 1313} файлов скопировано
horizontal_flip: {'SPIRAL': 1313, 'ELLIPTICAL': 1313, 'UNCERTAIN': 1313, 'MERGER': 1313} файлов скопировано
vertical_flip: {'SPIRAL': 1313, 'ELLIPTICAL': 1313, 'UNCERTAIN': 1313, 'MERGER': 1313} файлов скопировано
rotate_90_cw: {'SPIRAL': 1313, 'ELLIPTICAL': 1313, 'UNCERTAIN': 1313, 'MERGER': 1313} файлов скопировано
rotate_90_ccw: {'SPIRAL': 1313, 'ELLIPTICAL': 1313, 'UNCERTAIN': 1313, 'MERGER': 1313} файлов скопировано
rotate_15_cw: {'SPIRAL': 1313, 'ELLIPTICAL': 1313, 'UNCERTAIN': 1313, 'MERGER': 1313} файлов скопировано
rotate_15_ccw: {'SPIRAL': 1313, 'ELLIPTICAL': 1313, 'UNCERTAIN': 1313, 'MERGER': 1313} файлов скопировано
ИТОГОВОЕ РАСПРЕДЕЛЕНИЕ: {'SPIRAL': 10504, 'ELLIPTICAL': 10504, 'UNCERTAIN': 10504, 'MERGER': 10504}
Создание финальной папки с 42000 изображениями завершено


# Модель для обучения

In [ ]:
import os
import time 
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from sklearn.metrics import classification_report

DATASET_PATH = 'D:/nir/gc/final_mixed_data__with_merge'  
MODEL_SAVE_PATH = 'galaxy_classifier_mixed_merge_.keras' 

IMG_SIZE = (128, 128)
BATCH_SIZE = 32
EPOCHS = 20

if not os.path.exists(DATASET_PATH):
    raise FileNotFoundError(f"Указанная директория '{DATASET_PATH}' не найдена.")


datagen = ImageDataGenerator(
    rescale=1.0/255.0,
    validation_split=0.2
)


train_generator = datagen.flow_from_directory(
    directory=DATASET_PATH,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=True,
    subset='training'
)


val_generator = datagen.flow_from_directory(
    directory=DATASET_PATH,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False,
    subset='validation'
)


model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),  
    Dense(4, activation='softmax') 
])



model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)


model.summary()


start_time = time.time()


history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=EPOCHS,
    steps_per_epoch=train_generator.samples // BATCH_SIZE,
    validation_steps=val_generator.samples // BATCH_SIZE,
)


end_time = time.time()


training_time = (end_time - start_time) / 60  
print(f"\nВремя обучения: {training_time:.2f} минут")


val_generator.reset()
predictions = model.predict(val_generator, steps=val_generator.samples // BATCH_SIZE + 1)
predicted_classes = np.argmax(predictions, axis=1) 
true_classes = val_generator.classes  
class_labels = list(val_generator.class_indices.keys())  


report = classification_report(true_classes, predicted_classes, target_names=class_labels)
print("\nОтчёт о классификации:")
print(report)


model.save(MODEL_SAVE_PATH)
print(f"\nМодель сохранена в {MODEL_SAVE_PATH}")


In [24]:
import tkinter as tk
from tkinter import filedialog, messagebox
from PIL import Image, ImageTk
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image


models_3class = {
    "Модель с оригинальными изображениями (3 класса)": load_model("galaxy_classifier_original.keras"),
    "Модель с оригинальными изображениями + шум (3 класса)": load_model("galaxy_classifier_original_noise.keras")
}

models_4class = {
    "Модель с оригинальными изображениями (4 класса)": load_model("galaxy_classifier_merge.keras"),
    "Модель с оригинальными изображениями + шум (4 класса)": load_model("galaxy_classifier_merge_noise.keras")
}


class_labels_3 = ['ELLIPTICAL', 'SPIRAL', 'UNCERTAIN']
class_labels_4 = ['ELLIPTICAL', 'MERGER', 'SPIRAL', 'UNCERTAIN']


class GalaxyClassifierApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Galaxy Classifier")
        self.root.geometry("500x600")

        self.image_label = tk.Label(root)
        self.image_label.pack(pady=10)

        self.model_type = tk.StringVar(value="Модель с оригинальными изображениями (3 класса)")
        self.model_options = list(models_3class.keys()) + list(models_4class.keys())
        tk.OptionMenu(root, self.model_type, *self.model_options).pack()

        tk.Button(root, text="Загрузить изображение", command=self.load_image).pack(pady=5)
        self.result_text = tk.StringVar()
        tk.Label(root, textvariable=self.result_text, font=("Arial", 12)).pack(pady=10)

    def load_image(self):
        file_path = filedialog.askopenfilename()
        if not file_path:
            return

        img = Image.open(file_path).convert('RGB')
        img_resized = img.resize((128, 128))
        img_array = image.img_to_array(img_resized) / 255.0
        img_array = np.expand_dims(img_array, axis=0)

        model_name = self.model_type.get()
        if model_name in models_3class:
            model = models_3class[model_name]
            labels = class_labels_3
        else:
            model = models_4class[model_name]
            labels = class_labels_4

        prediction = model.predict(img_array)
        class_index = np.argmax(prediction)

        self.result_text.set(f"Предсказанный класс: {labels[class_index]}")

 
        display_img = img.resize((256, 256))
        tk_img = ImageTk.PhotoImage(display_img)
        self.image_label.configure(image=tk_img)
        self.image_label.image = tk_img


if __name__ == '__main__':
    root = tk.Tk()
    app = GalaxyClassifierApp(root)
    root.mainloop()